# Django 初始化

In [2]:
import django_setup

Development settings loaded
INSTALLED_APPS: ['django.contrib.admin', 'django.contrib.auth', 'django.contrib.contenttypes', 'django.contrib.sessions', 'django.contrib.messages', 'django.contrib.staticfiles', 'rest_framework', 'corsheaders', 'storages', 'apps.authentication', 'apps.files', 'apps.projects', 'apps.doc_analysis', 'apps.chat', 'django_filters', 'drf_spectacular', 'rest_framework_simplejwt.token_blacklist', 'django_celery_results', 'django_celery_beat']


INFO 2025-03-07 15:01:44,429 storage default_storage 的类型: COSStorage


Settings从哪里加载？: config.settings.development
项目根目录对么？: C:\Users\huiwa\Documents\_All_Projects\BidPilot_new\backend
文件存储settings对么？: apps.files.storage.COSStorage
文件default_storage对么？: COSStorage

已经安装的应用 Installed Apps 完整了么？:
- django.contrib.admin
- django.contrib.auth
- django.contrib.contenttypes
- django.contrib.sessions
- django.contrib.messages
- django.contrib.staticfiles
- rest_framework
- corsheaders
- storages
- apps.authentication
- apps.files
- apps.projects
- apps.doc_analysis
- apps.chat
- django_filters
- drf_spectacular
- rest_framework_simplejwt.token_blacklist
- django_celery_results
- django_celery_beat


In [2]:
#导入所有模型
from django.contrib.auth import get_user_model
from apps.files.models import FileRecord

from apps.doc_analysis.models import DocumentAnalysis


projects = Project.objects.all()  #事实上可以直接使用
files = FileRecord.objects.all()
analyses = DocumentAnalysis.objects.all()


数据库数据清理，只保留18501771516用户的数据


In [ ]:
# 删除18501771516用户以外的项目
deleted_count = Project.objects.exclude(creator__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他项目")


# 删除18501771516用户以外的文件
deleted_count = FileRecord.objects.exclude(owner__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他文件")

# 删除18501771516用户以外的分析
deleted_count = DocumentAnalysis.objects.exclude(created_by__phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他分析")



# <font color="red">Part 1 - 用户模型管理</font>


#### 1.1 contrib.auth模型引用 和 get_user_model方法 

In [11]:
# 模型引用import
from django.contrib.auth import get_user_model
User = get_user_model()

#### 1.2 User Model 常规字段 和 关联字段

In [4]:
# 用户模型所有字段
print("User")
print("-----------常规字段--------------")
for field in User._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in User._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"
        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in User._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")



User
-----------常规字段--------------
- password: CharField
- last_login: DateTimeField
- is_superuser: BooleanField
- id: UUIDField
- phone: CharField
- email: EmailField
- username: CharField
- wechat_id: CharField
- role: CharField
- is_active: BooleanField
- is_staff: BooleanField
- created_at: DateTimeField
- updated_at: DateTimeField

-----------反向关系字段-------------
- logentry_set: ManyToOneRel (来自 LogEntry)
- owned_files: ManyToOneRel (来自 FileRecord)
- created_projects: ManyToOneRel (来自 Project)
- created_analyses: ManyToOneRel (来自 DocumentAnalysis)
- confirmed_analyses: ManyToOneRel (来自 DocumentAnalysis)
- chat_sessions: ManyToOneRel (来自 ChatSession)
- outstandingtoken_set: ManyToOneRel (来自 OutstandingToken)

-----------多对多字段-------------
- groups: ManyToManyField
- user_permissions: ManyToManyField


#### 1.3 Creat, Get, Filter, Delete, Print 用户

In [7]:
#创建用户 - Create
User.objects.create_user(phone='18501771516', password='123456')

<User: 18501771516>

In [8]:
# 删除18501771516用户以外的用户
deleted_count = User.objects.exclude(phone='18501771516').delete()
print(f"已删除 {deleted_count[0]} 个其他用户")

已删除 0 个其他用户


In [10]:
# 获取用户 - Get
user = User.objects.get(phone='18501771516')
print(user)

18501771516


In [ ]:
# 过滤用户 - Filter
for user in User.objects.filter(phone='18501771516'):
    print(f"用户ID: {user.id}, 用户名: {user.username}")
    # 打印用户的所有字段值

    for field in User._meta.fields:
        print(f"  {field.name}: {getattr(user, field.name)}")
    print("---")

In [9]:
# 打印用户列表  .all()
users = User.objects.all()
for user in users:
    print(f"用户：{user.phone}," 
          f"项目数：{user.created_projects.all().count()}," 
          f"文件数：{user.owned_files.all().count()}",
          f"分析数：{user.created_analyses.all().count()}"
          )

用户：18501771516,项目数：0,文件数：0 分析数：0


In [ ]:
# 删除selected之外的所有用户 - exclude(条件).delete()
selected_phones = ['18501771516', '13800138000']
deleted_count = User.objects.exclude(phone__in=selected_phones).delete()
print(f"已删除 {deleted_count[0]} 个其他用户")


In [8]:
# 清空所有用户  .all().delete()
#delete_count = User.objects.all().delete()
#print(delete_count[0])

#### 1.4 related_fields 的使用 （参考字段） 

# <font color="red">Part 2 - 项目管理</font>


In [12]:
from apps.projects.models import Project, ProjectHistory, ProjectStage, BaseTask

In [19]:
# Project Model 所有字段
print("Project")
print("-----------常规字段--------------")
for field in Project._meta.fields:
    is_required = not field.blank  # 如果 blank 为 False，则为必填字段
    has_default = field.default is not None  # 如果 default 不为 None，则有默认值
    is_nullable = field.null  # 如果 null 为 True，则可以为空

    print(f"- {field.name}: {field.__class__.__name__}, "
          f"必填: {is_required}, "
          f"有默认值: {has_default}, "
          f"可为空: {is_nullable}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in Project._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"
        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in Project._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

Project
-----------常规字段--------------
- id: UUIDField, 必填: True, 有默认值: True, 可为空: False
- project_name: CharField, 必填: True, 有默认值: True, 可为空: False
- tenderee: CharField, 必填: True, 有默认值: True, 可为空: False
- bidder: CharField, 必填: False, 有默认值: True, 可为空: False
- project_type: CharField, 必填: True, 有默认值: True, 可为空: False
- bid_deadline: DateTimeField, 必填: False, 有默认值: True, 可为空: True
- status: CharField, 必填: True, 有默认值: True, 可为空: False
- is_urgent: BooleanField, 必填: True, 有默认值: True, 可为空: False
- current_active_stage: CharField, 必填: True, 有默认值: True, 可为空: False
- creator: ForeignKey, 必填: True, 有默认值: True, 可为空: False
- create_time: DateTimeField, 必填: False, 有默认值: True, 可为空: False
- last_update_time: DateTimeField, 必填: False, 有默认值: True, 可为空: False

-----------反向关系字段-------------
- files: ManyToOneRel (来自 FileRecord)
- stages: ManyToOneRel (来自 ProjectStage)
- project_histories: ManyToOneRel (来自 ProjectHistory)
- document_analyses: ManyToOneRel (来自 DocumentAnalysis)

-----------多对多字段--------

In [14]:
# ProjectStage 模型所有字段
print("ProjectStage")
print("-----------常规字段--------------")
for field in ProjectStage._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in ProjectStage._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"
        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in ProjectStage._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

ProjectStage
-----------常规字段--------------
- id: BigAutoField
- project: ForeignKey
- stage_type: CharField
- name: CharField
- stage_status: CharField
- description: TextField
- file_id: CharField
- progress: IntegerField
- remarks: TextField
- created_at: DateTimeField
- updated_at: DateTimeField
- metadata: JSONField

-----------反向关系字段-------------
- tasks: ManyToOneRel (来自 BaseTask)

-----------多对多字段-------------


In [ ]:
# 创建Project - Create
# 创新时 必填字段： 
Project.objects.create(creator = )

In [15]:
# 打印项目列表
projects = Project.objects.all()
for project in projects:
    print(f"项目编号:{project.project_code}",
          f"项目名：{project.project_name}," 
          f"创建者：{project.creator.phone}," 
          f"关联分析数：{project.document_analyses.all().count()}"
          )

In [16]:
# 清空所有上传的文件
#delete_count = Project.objects.all().delete()
#print(delete_count[0])

In [18]:
# 打印特定用户的所有项目的所有字段值
for project in Project.objects.filter(creator__phone='18501771516'):
    print(f"项目ID: {project.id}, 项目名: {project.project_name}")
    # 打印项目的所有字段值

    for field in project._meta.fields:
        print(f"  {field.name}: {getattr(project, field.name)}")
    print("---")



# <font color="red">Part 3 - 文件管理</font>


In [ ]:
# 文件所有字段
# 文件的所有常规字段
print("FileRecord")
print("-----------常规字段--------------")
for field in FileRecord._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in FileRecord._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"

        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in FileRecord._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

In [ ]:
# 打印项目列表
files = FileRecord.objects.all()
for file in files:
    print(f"文件ID:{file.id}",
          f"文件类型:{file.type}",
          f"文件名：{file.name}," 
          f"拥有者：{file.owner.phone}," 
          f"关联分析数：{file.document_analyses.all().count()}"
          )

In [18]:
# 清空所有上传的文件
#delete_count = FileRecord.objects.all().delete()
#print(delete_count[0])

In [19]:
# 打印特定用户的所有项目的所有字段值
for file in FileRecord.objects.filter(owner__phone='18501771516',name='Test 文本.docx'):
    print(f"文件ID: {file.id}, 文件名: {file.name}")
    # 打印文件的所有字段值


    for field in file._meta.fields:
        print(f"  {field.name}: {getattr(file, field.name)}")
    print("---")

<font color="red">将文件关联到项目</font>


In [17]:
#delete_count = FileProjectLink.objects.filter(project=project).delete()
#print(deleted_count[0])

# <font color=red>Part 4 - 分析管理 </font>

In [ ]:
# 文档分析的所有字段
# 常规字段
print("DocumentAnalysis")
print("-----------常规字段--------------")
for field in DocumentAnalysis._meta.fields:
    print(f"- {field.name}: {field.__class__.__name__}")

print("\n-----------反向关系字段-------------")
# 反向关系字段
for field in DocumentAnalysis._meta.get_fields():
    # 检查是否是反向关系字段
    if field.is_relation and field.auto_created:
        related_name = field.related_name or f"{field.name}_set"

        print(f"- {related_name}: {field.__class__.__name__} (来自 {field.related_model.__name__})")

print("\n-----------多对多字段-------------")
for field in DocumentAnalysis._meta.many_to_many:
    print(f"- {field.name}: {field.__class__.__name__}")

In [14]:
analyses = DocumentAnalysis.objects.all()

In [ ]:
print(analyses)

In [16]:
# 打印项目列表
analyses = DocumentAnalysis.objects.all()
for analysis in analyses:
    print(f"分析号:{analysis.id}\n")

In [22]:
# 打印项目列表
analyses = DocumentAnalysis.objects.all()
for analysis in analyses:
    print(f"分析号:{analysis.id}\n",
          f"分析名称：{analysis.title}\n",
          f"分析所在项目：{analysis.project.project_name}\n", 
          f"分析的文件：{analysis.file_record.name}\n" ,
         # f"分析的阶段：{analysis.status}\n",
         # f"分析的问题：{analysis.analysis_questions}\n",
         # f"分析结果：{analysis.analysis_result}\n",
         # f"分析创建者：{analysis.created_by.phone}\n",
         # f"分析用时：{analysis.processing_time}\n",
          )

In [ ]:
delete_count = DocumentAnalysis.objects.all().delete()
print(delete_count[0])